##### OpenEarthMap Land Cover Mapping Few-Shot Challenge Baseline

In [ ]:
# Clone github repository and install the requirements libraries
# !git clone https://github.com/cliffbb/OEM-Fewshot-Challenge.git
# !cd OEM-Fewshot-Challenge
# !pip install -r requirements.txt

In [ ]:
# import the necessary requirements
import os
import torch.multiprocessing as mp

from src.utils import find_free_port, load_cfg_from_cfg_file
from src.test import main_worker

In [ ]:
# Load config file
# Set the 'data_root' and 'pretrained' path in the the config file `eom.yaml` 
cfg_args = load_cfg_from_cfg_file('config/oem.yaml') 

In [ ]:
# Set the environment
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in cfg_args.gpus)
os.environ['OPENBLAS_NUM_THREADS'] = '1'
world_size = len(cfg_args.gpus)
distributed = world_size > 1
assert not distributed, 'Testing should not be done in a distributed way'
cfg_args.distributed = distributed
cfg_args.port = find_free_port()

In [ ]:
# Check for torch multiprocessing 
try:
    mp.set_start_method('spawn')
except RuntimeError:
    pass

# Call the `main_worker` function to run the few-shot evaluation
mp.spawn(main_worker, args=(world_size, cfg_args ), nprocs=world_size, join=True)